In [15]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [16]:
df_train=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")
oil=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
hol=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
stores=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
transact=pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
hol = hol.rename(columns={"type": "type_hol"})
stores=stores.rename(columns={'type':'type_store'})

## Combiinig/edditing/adding the data

In [17]:
combined=pd.merge(pd.merge(pd.merge(df_train,oil,on='date',how='outer'),hol,on='date',how='outer'),stores,on='store_nbr', how='outer')

In [18]:
combined['year']=combined['date'].apply(lambda x: x.split('-')[0]).astype(int)
combined['month']=combined['date'].apply(lambda x: x.split('-')[1]).astype(int)
combined['day']=combined['date'].apply(lambda x: x.split('-')[2]).astype(int)
combined = combined.rename(columns={"dcoilwtico": "oil"})
combined = combined.rename(columns={'description': "hol_name"})
combined = combined.rename(columns={'locale': "loc/nat"})

In [19]:
import calendar

def date_to_day(year,month,day):
    d = calendar.weekday(year, month, day)
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    return days[d]
# Map the weekday number to the day name
 
date_to_day(1917,3,4)

'Sunday'

In [21]:
combined['day_of_the_week']=combined.apply(lambda row: date_to_day(row['year'], row['month'], row['day']), axis=1)
combined = combined.merge(
   transact[['date', 'store_nbr', 'transactions']],
    on=['date', 'store_nbr'],
    how='left'
)

In [22]:
 combined=combined[[ 'date','year', 'month', 'day','day_of_the_week', 'store_nbr', 'family', 'sales', 'transactions','onpromotion', 'oil',
       'type_hol', 'loc/nat', 'hol_name', 'transferred', 'city',
       'state', 'type_store', 'cluster']]

In [23]:
 combined.to_csv('/kaggle/working/combined.csv', index=False)  # Save without the index column

## Cheking the type of store:

In [24]:
 for typ in  set(combined['type_store']):
    filtered_df=combined.loc[combined['type_store']==typ][['family','sales']]
    family_sales = filtered_df.groupby('family')['sales'].sum().astype(int)
    top_3=family_sales.sort_values(ascending=False)[:3]
    print('Type= ',typ,'\n top 3=', top_3,'\n')

Type=  D 
 top 3= family
GROCERY I    101766057
BEVERAGES     71050081
PRODUCE       54039577
Name: sales, dtype: int64 

Type=  A 
 top 3= family
GROCERY I    107869009
BEVERAGES     73335263
PRODUCE       47208554
Name: sales, dtype: int64 

Type=  E 
 top 3= family
GROCERY I    20680983
BEVERAGES    14602000
CLEANING      5739097
Name: sales, dtype: int64 

Type=  B 
 top 3= family
GROCERY I    54953102
BEVERAGES    29572844
CLEANING     15450788
Name: sales, dtype: int64 

Type=  nan 
 top 3= Series([], Name: sales, dtype: int64) 

Type=  C 
 top 3= family
GROCERY I    65558144
BEVERAGES    33103352
CLEANING     21319073
Name: sales, dtype: int64 



## Checking that transactions are the same for each day for each store nbhd 

In [32]:
combined.loc[combined['date']=='2013-01-01'].loc[combined['store_nbr']==25] 

,date,year,month,day,day_of_the_week,store_nbr,family,sales,transactions,onpromotion,oil,type_hol,loc/nat,hol_name,transferred,city,state,type_store,cluster
1357488,2013-01-01,2013,1,1,Tuesday,25.0,AUTOMOTIVE,0.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357489,2013-01-01,2013,1,1,Tuesday,25.0,BABY CARE,0.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357490,2013-01-01,2013,1,1,Tuesday,25.0,BEAUTY,2.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357491,2013-01-01,2013,1,1,Tuesday,25.0,BEVERAGES,810.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357492,2013-01-01,2013,1,1,Tuesday,25.0,BOOKS,0.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357493,2013-01-01,2013,1,1,Tuesday,25.0,BREAD/BAKERY,180.589000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357494,2013-01-01,2013,1,1,Tuesday,25.0,CELEBRATION,0.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357495,2013-01-01,2013,1,1,Tuesday,25.0,CLEANING,186.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357496,2013-01-01,2013,1,1,Tuesday,25.0,DAIRY,143.000000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
1357497,2013-01-01,2013,1,1,Tuesday,25.0,DELI,71.090000,770.0,0.0,NaN,Holiday,National,Primer dia del ano,False,Salinas,Santa Elena,D,1.0
